In [47]:
# The 9 Heegner Numbers are 1, 2, 3, 7, 11, 19, 43, 67 and 163. For all those, extended sieving works
# based on the chosen h, the algorithm will adapt itself to create the right function

d = 163

def f(x, y =1):
    """ General method that returns the value of the function x² + xy + Cy² or x² + C y², depending on the value of h """
    if d % 4 == 3:
        C = int((d+1)/4)
        return x ** 2 + x * y + C * y ** 2
    else:
        # this covers the case when d equals 1 or 2
        C = d
        return x ** 2 + C * y ** 2

def is_prime(n):
    """ Checks if a number is prime by (regular) trial division """
    if n == 2 or n == 3: return True
    if n < 2 or n % 2 == 0: return False
    if n < 9: return True
    if n % 3 == 0: return False
    r = int(n ** 0.5)
    t = 5
    while t <= r:
        if n % t == 0: return False
        if n % (t + 2) == 0: return False
        t += 6
    return True



def find_x_values_composites(x_max):
    """ 
        Finds all the x-values of the composites of f(x) defined above, using the following properties. 
        When f(x) = p*q, 
            f(x+k*q) is divisible by q (and another divisor, f(x+q)/q) for each k
        Goes first through all x and then will continue to create more composites until everything is treated. 
        The cutoff value makes sure that the sieve stops after certain cutoff, namely x_max.  
    """    
    err_x_pos = [] # list to track the x-value of all the composites
    err_q = []
    counter = 0    
    
    for x in range(-round(1.2*x_max**0.5), round(1.2*x_max**0.5)): # only need to evaluate x up to a bit more than sqrt(x) as the f(x) is quadratic
        if (x+f(x)) < x_max: # as long as the first composite, on f(x)+x is not too big, search for composites
            if f(x) != 1 and f(x) != f(x+f(x)): # For some small h, we need to filter out some edge cases 
                err_x_pos.append(x+f(x)) # add the first x-position to treat to the error list
                err_q.append(f(x)) # add the divisor q, equaling f(x) to the error list

            # now start to find more composites, until done
            while counter < len(err_x_pos): # while there are still composites to treat
                k = 1 # start a counter for k to generate all the composites on x + k*q
                while (err_x_pos[counter] + k * err_q[counter]) < x_max: # add those composites only that don't exceed the cutoff value
                    pos = err_x_pos[counter] + k * err_q[counter] 
                    q = f(pos)//err_q[counter] 
                    
                    err_x_pos.append(pos) 
                    err_q.append(q)
                    k += 1
                counter+=1
    return err_x_pos


   
def get_primes(composites_x, x_max):
    """ Calculates the primes """
    primes = []
    for i in range(1, x_max):
        if not (i in composites_x):
            primes.append(f(i))
    return primes

def check_primes(composites_x, x_max):
    """ Checks if there are any composites missed """
    missed = []
    for i in range(1, x_max):
        if not (i in composites_x):
            if not(is_prime(f(i))):
                missed.append(i)
    return missed

def check_composites(composites_x):
    """ Checks if there are any composites detected that are actually primes """
    errors = []
    for composite in composites_x:
        if is_prime(f(composite)):
            errors.append(composite)
    return errors

In [48]:
cutoff_value = 150
print(f"Sieving the polynomial using d={d} up until {cutoff_value}")
composites_x_values = find_x_values_composites(cutoff_value)
composites_x_values.sort()
print('Detected composites (x-position):')
print(composites_x_values)


print("Evaluating results....")
missed = check_primes(composites_x_values, cutoff_value)
if missed:    
    print('There are still composites not found (x-position):')
    missed.sort()
    print(missed)
else:
    print("The sieve did not miss any composites")
    
errors = check_composites(composites_x_values)
if errors:
    print('There are crossed out primes (x-position):')
    errors.sort()
    print(errors.sort())
else:
    print("There are no primes mistakenly crossed out")


Sieving the polynomial using d=163 up until 150
Detected composites (x-position):
[40, 41, 41, 44, 44, 49, 49, 56, 56, 65, 65, 76, 76, 81, 82, 84, 87, 89, 89, 91, 96, 102, 104, 104, 109, 117, 121, 121, 122, 123, 126, 127, 130, 136, 138, 140, 140, 143, 147]
Evaluating results....
The sieve did not miss any composites
There are no primes mistakenly crossed out
